In [3]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

df_web_data = pd.read_csv('../02_DATA_LIMPIO/df_web_data_combinado_limpio.csv')
df_expanded = pd.read_csv('../02_DATA_LIMPIO/df_final_expanded_limpio.csv')

In [10]:
# Cargar los archivos
df_web_data = pd.read_csv('../02_DATA_LIMPIO/df_web_data_combinado_limpio.csv')
df_expanded = pd.read_csv('../02_DATA_LIMPIO/df_final_expanded_limpio.csv')

# Mostrar columnas de cada archivo
print("Columnas en df_web_data:")
print(df_web_data.columns)

print("\nColumnas en df_expanded:")
print(df_expanded.columns)

Columnas en df_web_data:
Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time'], dtype='object')

Columnas en df_expanded:
Index(['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth', 'Variation',
       'total_tenure_months'],
      dtype='object')


In [11]:
# Hacemos un merge de los datos usando 'client_id' como clave común
df_merged = pd.merge(df_web_data, df_expanded, on='client_id', how='inner')

In [12]:
# Filtro los grupos según la columna 'Variation'
grupo_control = df_merged[df_merged['Variation'] == 'control']
grupo_prueba = df_merged[df_merged['Variation'] == 'test']

In [13]:
# Calculo finalizaciones (process_step == 'confirm') y visitas únicas
finalizados_control = grupo_control[grupo_control['process_step'] == 'confirm'].shape[0]
finalizados_prueba = grupo_prueba[grupo_prueba['process_step'] == 'confirm'].shape[0]
n_control = grupo_control['visit_id'].nunique()
n_prueba = grupo_prueba['visit_id'].nunique()

In [14]:
print(f"Finalizaciones - Control: {finalizados_control}, Prueba: {finalizados_prueba}")
print(f"Visitas únicas - Control: {n_control}, Prueba: {n_prueba}")

Finalizaciones - Control: 17498, Prueba: 25716
Visitas únicas - Control: 32181, Prueba: 37122


**Prueba estadística y evaluar el umbral**

In [15]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

In [16]:
# Prueba de proporciones Z-test
counts = np.array([finalizados_prueba, finalizados_control])  # Finalizaciones
nobs = np.array([n_prueba, n_control])  # Total de visitas únicas

In [17]:
z_stat, p_value = proportions_ztest(count=counts, nobs=nobs, alternative='two-sided')

In [18]:
# Calculo las tasas
tasa_control = finalizados_control / n_control
tasa_prueba = finalizados_prueba / n_prueba
diferencia_tasa = (tasa_prueba - tasa_control) * 100

In [19]:
print(f"Estadístico Z: {z_stat:.2f}")
print(f"Valor p: {p_value:.4f}")
print(f"Tasa de finalización - Control: {tasa_control:.2%}")
print(f"Tasa de finalización - Prueba: {tasa_prueba:.2%}")
print(f"Diferencia de tasas: {diferencia_tasa:.2f}%")

Estadístico Z: 40.38
Valor p: 0.0000
Tasa de finalización - Control: 54.37%
Tasa de finalización - Prueba: 69.27%
Diferencia de tasas: 14.90%


In [20]:
# Evaluar el umbral del 5%
umbral = 5
if diferencia_tasa > umbral:
    print("La diferencia de tasas supera el umbral del 5%.")
else:
    print("La diferencia de tasas no supera el umbral del 5%.")

La diferencia de tasas supera el umbral del 5%.


In [21]:
# Creo un df con los resultados resumidos
resultados = pd.DataFrame({
    'Grupo': ['Control', 'Prueba'],
    'Tasa de Finalización': [tasa_control * 100, tasa_prueba * 100],
    'Finalizaciones': [finalizados_control, finalizados_prueba],
    'Total Visitas': [n_control, n_prueba]
})

In [23]:
resultados.to_csv('../02_DATA_LIMPIO/resultados_finalizacion.csv', index=False)

**Comparar tasas por género**

In [27]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind

In [28]:
# Crea una columna binaria 'finalizo' para identificar finalizaciones
df_merged['finalizo'] = df_merged['process_step'] == 'confirm'

In [29]:
tabla_gendr = pd.crosstab(df_merged['gendr'], df_merged['finalizo'])

# Prueba chi-cuadrado
chi2, p_gendr, dof, expected = chi2_contingency(tabla_gendr)

print("Comparación por Género:")
print(tabla_gendr)
print(f"Chi-cuadrado: {chi2:.2f}, p-valor: {p_gendr:.4f}")

Comparación por Género:
finalizo  False  True 
gendr                 
F         90569  13721
M         93011  15002
U         94393  14491
X             8      0
Chi-cuadrado: 28.70, p-valor: 0.0000


In [30]:
# Dividimos la edad en grupos
df_merged['edad_grupo'] = pd.cut(df_merged['clnt_age'], bins=[0, 30, 50, 70, 100], labels=["<30", "30-50", "50-70", "70+"])

In [31]:
# Calculo tasas de finalización por grupo de edad
tasas_edad = df_merged.groupby('edad_grupo')['finalizo'].mean() * 100
print("Tasas de finalización por grupo de edad:")
print(tasas_edad)

Tasas de finalización por grupo de edad:
edad_grupo
<30      14.126309
30-50    14.010055
50-70    12.937783
70+      12.111657
Name: finalizo, dtype: float64


In [33]:
# ANOVA para comparar grupos de edad
grupos_edad = [df_merged[df_merged['edad_grupo'] == g]['finalizo'].astype(int) for g in df_merged['edad_grupo'].unique()]
from scipy.stats import f_oneway
anova_edad = f_oneway(*grupos_edad)
print(f"ANOVA para edad: F={anova_edad.statistic:.2f}, p-valor={anova_edad.pvalue:.4f}")

ANOVA para edad: F=38.34, p-valor=0.0000


In [34]:
# Crear grupos de número de cuentas
df_merged['num_accts_grupo'] = pd.cut(df_merged['num_accts'], bins=[0, 2, 5, 10, 20], labels=["1-2", "3-5", "6-10", "11+"])

# Calcular tasas
tasas_cuentas = df_merged.groupby('num_accts_grupo')['finalizo'].mean() * 100
print("Tasas de finalización por número de cuentas:")
print(tasas_cuentas)

# Prueba ANOVA
grupos_cuentas = [df_merged[df_merged['num_accts_grupo'] == g]['finalizo'].astype(int) for g in df_merged['num_accts_grupo'].unique()]
anova_cuentas = f_oneway(*grupos_cuentas)
print(f"ANOVA para número de cuentas: F={anova_cuentas.statistic:.2f}, p-valor={anova_cuentas.pvalue:.4f}")


Tasas de finalización por número de cuentas:
num_accts_grupo
1-2     12.988470
3-5     15.088661
6-10    20.975610
11+           NaN
Name: finalizo, dtype: float64
ANOVA para número de cuentas: F=109.19, p-valor=0.0000


**Conclusiones**

Edad:

- Los usuarios menores de 30 años y entre 30-50 años tienen tasas de finalización más altas.
- Los usuarios mayores de 50 años, especialmente los 70+, tienen tasas de finalización más bajas.

Número de cuentas:
- Los usuarios con 6-10 cuentas tienen una tasa de finalización significativamente más alta (20.98%) en comparación con otros grupos.
- Los usuarios con 3-5 cuentas también muestran mejores tasas que aquellos con solo 1-2 cuentas.
